# Install

In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

!pip install bert-tensorflow cort
!pip install git+https://github.com/guillaumegenthial/tf_metrics.git
  
%load_ext tensorboard

import nltk
nltk.download('wordnet')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
  Cloning https://github.com/guillaumegenthial/tf_metrics.git to /tmp/pip-req-build-uja1807q
  Running command git clone -q https://github.com/guillaumegenthial/tf_metrics.git /tmp/pip-req-build-uja1807q
  Created wheel for tf-metrics: filename=tf_metrics-0.0.1-cp36-none-any.whl size=7694 sha256=ceea43f61ac03cab941bbb777e9f58bd6a09bca2d6242324ee2c525690468f26
  Stored in directory: /tmp/pip-ephem-wheel-cache-wiuhk6b6/wheels/da/6c/c8/663ef339a0666590dc53bd13bab86643a1f9c35b26742d7876
Successfully built tf-metrics
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Parameters

In [0]:
root = r"/content/gdrive/My Drive/Puc/Projeto Final"
test_file = f"{root}/Datasets/conll/conll-2012-test.conll"
predicted_file = f"{root}/Datasets/predicted_colab.txt"


# Modules

## Boilerplate

### Saver

In [0]:
"""

This module is responsible for generating the output files for a given prediction. It will
use the original files to keep the format compatible
"""
import os


class LineInfo:
    """
    Class representing all operations (open/close brackets) for all clusters in a given line
    """

    def __init__(self):
        # Cluster id => Operation
        # Operations can be
        # 1 -> Open bracket
        # -1 -> Close bracket
        # 0 -> Open AND Close bracket
        self.operations = {}

    def add_operation(self, cluster_id, operation):
        """
        Adds the operation for that cluster_id in the current line
        :param cluster_id:
        :param operation:
        """
        if cluster_id not in self.operations:
            self.operations[cluster_id] = 0

        # By the definition, adding a close to a open leads to a open/close operation
        self.operations[cluster_id] += operation

    def __str__(self):
        """
        Method to translate this object into a string. In this implementation it will be translated
        into the CoNLL format for the mention coreference notation
        :return:
        """
        per_cluster = [self._to_simple_oper(k, v) for k, v in self.operations.items()]
        return "|".join(per_cluster)

    @staticmethod
    def _to_simple_oper(cluster_id, operation):
        """
        Translates the operation from number to string using open/close brackets.
        For example:
        '1', 1 => (1
        '2', -1 => 2)
        '3', 0 => (3)

        :param cluster_id:
        :param operation: integer representing the operation (1,-1,0)
        :return: string
        """
        output = str(cluster_id)
        if operation >= 0:
            output = "(" + output
        if operation <= 0:
            output += ")"

        return output


class Document:
    """
    Class representing a document and its mention clusters. This is the object that an algorithm should
    generate to allow the framework to save it in the proper CoNLL format.
    """

    def __init__(self, name, first_line_idx):
        self.name = name
        # Each cluster is a dictionary that maps start -> end for each mention in that cluster. Start/end are the
        # original document line number
        self.clusters = []

        # Line number => line info
        self.line_info = {}
        self.first_line_idx = first_line_idx
        self.next_line = 0
        self.next_line_generator = self._get_next_line()

    def add_cluster(self, cluster):
        """
        Adds a cluster to the document
        :param cluster: a dictionary that maps start_line -> [end_line] for each mention
        """
        self.clusters.append(cluster)
        cluster_id = len(self.clusters)

        # Adding one operation for key and another for value
        for key, values in cluster.items():
            self._add_operation(cluster_id, key, 1)
            for v in values:
                self._add_operation(cluster_id, v, -1)

    def _add_operation(self, cluster_id, line_number, operation):
        """
        Adds a single operation to the line
        :param cluster_id: generic cluster id
        :param line_number:
        :param operation: +1 -> open, -1 -> close
        :return:
        """
        if line_number not in self.line_info:
            self.line_info[line_number] = LineInfo()

        self.line_info[line_number].add_operation(cluster_id, operation)

    def _get_line_operations(self, line_number):
        """
        Gets the operation for a given line number
        :param line_number:
        :return:
        """
        if self.next_line < line_number:
            try:
                self.next_line = next(self.next_line_generator)
            except StopIteration:
                # If this exception occurs, it means that there are no more
                # lines to be generated. All following lines must be marked with -
                self.next_line += 1000

        if line_number == self.next_line:
            return str(self.line_info[line_number])

        return "-"

    def _get_next_line(self):
        """
        Generator for line numbers. Iterates over added lines information
        """
        for line_number in sorted(self.line_info):
            yield line_number

    def get_line(self, counter):
        return self.name + " " + self._get_line_operations(counter - self.first_line_idx)


def save_document(original_path, output_path, document):
    """
    Saves the document object to the output_path. It reads the original file to keep the common lines

    :param original_path:
    :param output_path:
    :param document: a Document object
    """
    with _open_original_file(original_path, document.name) as o:
        with _open_destination_file(output_path, document.name) as d:
            counter = 0
            for line in o:
                counter += 1

                if _write_as_is(line):  # Some lines are identical in both documents
                    d.write(line)
                else:  # Others are simply the name and the open/close mention (or a dash)
                    d.write(document.name + " " + document._get_line_operations(counter) + "\n")


def save_documents(in_file, out_file, docs):
    doc_iterator = iter(docs)
    with open(in_file, "r") as entrada, open(out_file, "w") as saida:
        counter = 0
        current_doc = next(doc_iterator)
        print(f"Writing doc {current_doc.name}")
        next_doc = next(doc_iterator)

        for line in entrada:
            counter += 1
            if _write_as_is(line):  # Some lines are identical in both documents
                saida.write(line)
            else:  # Others are simply the name and the open/close mention (or a dash)
                if (next_doc is not None) and (next_doc.first_line_idx <= counter):
                    current_doc = next_doc
                    print(f"Writing doc {current_doc.name}")
                    next_doc = next(doc_iterator)
                    print("Moving to next document")

                saida.write(current_doc.get_line(counter) + "\n")


def _open_original_file(original_path, name):
    """
    Opens (read) the gold_conll file associated with the given file
    :param original_path: root path (generally the 'annotation' folder)
    :param name: file name
    :return: a file object
    """
    return open("{}/{}.gold_conll".format(original_path, name), "r", encoding="utf8")


def _open_destination_file(output_path, name):
    """
    Opens (write) the file to be used as output. Will create any folder structe needed

    :param output_path:
    :param name: file
    :return: a file object
    """
    try:
        # This will create the folder structure to save the file
        os.makedirs(os.path.join(output_path, *name.split("/")[:-1]))
    except FileExistsError:
        pass  # Do nothing - folder structure already exists
    return open("{}/{}.output".format(output_path, name), "w")


def _write_as_is(line):
    """
    Checks if a line should be copied to the final document.
    The header, footer and blank lines are the only ones in this case

    :param line:
    :return: True if blank, header or footer. False otherwise
    """
    if line == "\n" or "#begin" in line or "#end" in line:
        return True
    return False


### mentions

In [0]:
"""
Mentions Module

This module will handle the parse of the files and creation of a MentionsPair class that will be used to generate
the output files.
"""

import difflib
import re

from tqdm import tqdm

CONLL_DOC_ID_COLUMN = 0
CONLL_PART_NUM_COLUMN = 1
CONLL_WORD_SEQ = 2
CONLL_WORD_COLUMN = 3
CONLL_POS_COLUMN = 4
CONLL_PARSE_BIT_COLUMN = 5
CONLL_LEMMA_COLUMN = 6
CONLL_FRAMESET_ID_COLUMN = 7
CONLL_SENSE_COLUMN = 8
CONLL_SPEAKER_COLUMN = 9
CONLL_NAMED_COLUMN = 10


class Mention:
    """
    This class represents a mention with all its features. It can be extended if more information is needed
    """

    def __init__(self, m_id, words, start_pos, end_pos):
        self.words = words
        self.mention_id = m_id
        self.doc_id = m_id.split('_')[0]
        self.mention = " ".join(words)
        self.first_word = words[0]
        self.last_word = words[-1]
        self.start_pos = start_pos
        self.end_pos = end_pos

    def compare(self, other):
        start_diff = self.start_pos - other.start_pos
        if start_diff != 0:
            return start_diff
        return self.end_pos - other.end_pos

    def __str__(self):
        return f"Mention ({self.doc_id},{self.start_pos},{self.end_pos})"


class MentionPair:
    """
    This class represents a pair of mentions. It can be extended if more information is needed
    """

    def __init__(self, mention1, mention2):
        self.mention1 = mention1
        self.mention2 = mention2
        self.coref = mention1.mention_id == mention2.mention_id

    def get_info_vector(self):
        """
        Return the information to enable the recovery of the original mention pair
        :return: [start/end position for both mentions]
        """
        return [self.mention1.start_pos, self.mention1.end_pos, self.mention2.start_pos, self.mention2.end_pos]


def check_usable_pairs(mention_list, i, j):
    """
    Default implementation for checking if two mentions should be considered as a pair. These pairs are not necessarily
    correferences
    :param mention_list: list of mentions
    :param i: first index
    :param j: second index
    :return: True if mention i and mention j should be paired
    """
    if mention_list[i].doc_id != mention_list[j].doc_id:
        return False
    if mention_list[i].mention_id == mention_list[j].mention_id:
        return True
    # Skipping some indexes so not all pairs will be generated
    if j % 2 == 0 or j % 3 == 0 or j % 5 == 0 or j % 7 == 0 or j % 11 == 0:
        return False
    return True


def build_mention_list(train_list, fill_information=None):
    """
    Build a list of dictionaries with the information about each mention. The mentions are not yet grouped

    :param train_list: list of lines in the document
    :param fill_information: function to add more information into the cluster
    :return:
    """
    mentions = []
    cluster_start, start_pos, cluster_end, end_pos = _get_mention(train_list)
    mention_cluster = _create_mention_cluster_list(cluster_start, start_pos, cluster_end, end_pos)
    for m in tqdm(mention_cluster, desc="mentions"):
        m_id, start_pos, end_pos = m

        mention_words = get_mention_words(train_list, start_pos, end_pos)
        mention = Mention(m_id, mention_words, start_pos, end_pos)

        # Building features
        mention.mention_start = start_pos
        mention.mention_end = end_pos
        mention.pre_words = _get_preceding_words(train_list, start_pos)
        mention.next_words = _get_next_words(train_list, end_pos)
        mention.mention_sentence = _mention_sentence(train_list, start_pos)
        mention.speaker = train_list[start_pos - 1].split()[CONLL_SPEAKER_COLUMN]

        # This will allow external info to be added
        if fill_information:
            fill_information(mention)

        mentions.append(mention)

    mentions = sorted(mentions, key=lambda k: k.mention_start)
    mentions = _check_mention_contain(mentions)
    mentions = _get_index(mentions)
    return mentions


def _add_extra_pair_info(mention_pair_list, train_list, increment_mention_pair=None):
    """
    Adds distance information about the mention pairs.
    'overlap' : True (1) if the second element overlaps the first
    'speaker' : True (1) if both mentions have the same speaker
    'mention_exact_match' : True(1) if both mentions are from the same sentence
    'mention_partial_match' : True(1) if the sentences are similar

    :param mention_pair_list: list of pair of mentions
    :param train_list: list of lines in the document
    :param increment_mention_pair: function to add more information to each pair
    :return:
    """
    for p in mention_pair_list:
        p.overlap = p.mention1.overlap == p.mention2.mention_id
        p.speaker = p.mention1.speaker == p.mention2.speaker
        p.mention_exact_match = p.mention1.mention == p.mention2.mention

        seq = difflib.SequenceMatcher(None, p.mention1.mention, p.mention2.mention)
        score = seq.ratio()
        p.mention_partial_match = score > 0.6

        if increment_mention_pair:
            increment_mention_pair(p, train_list)

    return mention_pair_list


def _get_mention(train_list):
    """
    Creates four auxiliary lists with IDs and positions. These lists are not completely related to each other. It will
    simply gather the open/close parenthesis in the order it appears in the document.
    Check the formatting of a CoNLL document in the site

        * Cluster start: ID of the opening parenthesis cluster
        * Start pos: line of starting position
        * Cluster end: ID of the closing parenthesis cluster (
        * End pos: line

    :param train_list: List of lines of the file
    :return: cluster_start, start_pos, cluster_end, end_pos
    """
    cluster_start = []
    start_pos = []
    cluster_end = []
    end_pos = []
    i = 1
    for line in train_list:
        if line == '\n' or line == '-':  # Ignore empty lines
            i += 1
            continue
        part_number = line.split()[CONLL_PART_NUM_COLUMN]  # Part number column
        coref_col = line.split()[-1]  # Coref column
        for j in range(len(coref_col)):  # There can be more than one parenthesis. Tracking all of them
            if coref_col[j] == '(':
                cluster_start.append((str(part_number) + '_' + re.findall(r'\d+', coref_col[j + 1:])[0]))
                start_pos.append(i)
            if coref_col[j] == ')':
                cluster_end.append((str(part_number) + '_' + re.findall(r'\d+', coref_col[:j])[-1]))
                end_pos.append(i)
        i += 1
    return cluster_start, start_pos, cluster_end, end_pos


def _create_mention_cluster_list(cluster_start, start_pos, cluster_end, end_pos):
    """
    Builds a list of mentions. One mention per item. The clusters are not yet grouped
     * First position is the id = [document_cluster]
     * Second position is the starting line
     * Third position in the ending line (counting from header).
     Use function get_mention to build the lists properly.

    :param cluster_start: List of IDs of starting cluster IDs (opening parenthesis)
    :param start_pos: List of positions for the starting cluster ID
    :param cluster_end: List of IDs for ending cluster (closing parenthesis)
    :param end_pos: List of positions for the ending cluster ID
    :return: List with three items and length of the same size of the input lists
    """
    cluster_start_end_list = []
    for start, pos in zip(cluster_start, start_pos):  # Join ID and position
        cluster = [start, pos]
        i = 0
        for i in range(len(cluster_end)):  # Search where is the closing parenthesis
            if cluster_end[i] == start:
                cluster.append(end_pos[i])  # Found it. Add to the pairing
                break
        del cluster_end[i]  # Remove from original list. This will prevent from being used again
        del end_pos[i]
        cluster_start_end_list.append(cluster)
    return cluster_start_end_list


def get_mention_words(train_list, pos1, pos2):
    """
    Gets the list of words between these lines
    :param train_list: lines of the document
    :param pos1: initial line
    :param pos2: final line
    :return: List of all words
    """
    mention = []
    for line_no in range(pos1 - 1, pos2):
        word = train_list[line_no].split()[CONLL_WORD_COLUMN]
        mention.append(word)
    return mention


def _get_preceding_words(train_list, pos, max_words=5):
    """
    Get the previous max_words in the document (if they exists)
    :param train_list: list of lines
    :param pos: word position (numer of the line)
    :param max_words: max words to look ahead
    :return:
    """
    word_part = train_list[pos - 1].split()[CONLL_PART_NUM_COLUMN]
    num_words = 0
    word = []
    for i in range(2, pos):
        if train_list[pos - i] != '\n':
            doc_id = train_list[pos - i].split()[CONLL_DOC_ID_COLUMN]  # Checking if begin/end of document was reached
            if doc_id == '#begin' or doc_id == '#end':
                break
            part_no = train_list[pos - i].split()[CONLL_PART_NUM_COLUMN]
            if part_no == word_part:  # Will add only words for the same part
                word.append(train_list[pos - i].split()[CONLL_WORD_COLUMN])
                num_words += 1
            if num_words == max_words:
                break
    return word


def _get_next_words(train_list, pos, max_words=5):
    """
    Get the next max_words in the document (if they exists)
    :param train_list: list of lines
    :param pos: word position (numer of the line)
    :param max_words: max words to look ahead
    :return:
    """
    pos = pos - 1
    word_part = train_list[pos].split()[CONLL_PART_NUM_COLUMN]
    num_words = 0
    word = []
    for i in range(1, len(train_list) - pos):
        if train_list[pos + i] != '\n':
            doc_id = train_list[pos + i].split()[CONLL_DOC_ID_COLUMN]  # Checking if begin/end of document was reached
            if doc_id == '#begin' or doc_id == '#end':
                break
            part_no = train_list[pos + i].split()[CONLL_PART_NUM_COLUMN]
            if part_no == word_part:
                word.append(train_list[pos + i].split()[CONLL_WORD_COLUMN])
                num_words += 1
            if num_words == max_words:
                break
        i += 1
    return word


def _mention_sentence(train_list, pos):
    """
    Gets the sentence that contains the mention in this line
    :param train_list: list of lines
    :param pos: line of reference
    :return: string with sentence
    """
    pos = pos - 1
    i = 1
    end = 0
    while True:
        if train_list[pos - i] == '\n':
            start = pos - i
            break
        if train_list[pos - i].split()[CONLL_DOC_ID_COLUMN] == '#begin':
            start = pos - i
            break
        i += 1
    start += 2
    i = 1

    while True:
        if train_list[pos + i] == '\n':
            end = pos + i
            break
        if train_list[pos + i].split()[CONLL_DOC_ID_COLUMN] == '#end':
            start = pos + i
            break
        i += 1
    sentence = get_mention_words(train_list, start, end)
    return " ".join(sentence)


def _check_mention_contain(mention_list):
    """
    Adds information about mentions containing/overlapping other mentions. The difference between contain and overlap
    will be defined by the end position. In this context, overlapping is not reflexive.
    The keys 'contained'/'overlap' will have the id of the container/overlapped mention or False if it is not contained

    :param mention_list: list of dictionaries, one item per mention. Each item must have the keys:
        mention_start. mention_end and id

    :return: the modified list
    """
    for i in range(0, len(mention_list)):
        start = mention_list[i].mention_start
        end = mention_list[i].mention_end

        for j in range(0, len(mention_list)):
            c_start = mention_list[j].mention_start
            c_end = mention_list[j].mention_end

            if c_start == start and c_end == end:
                continue

            if c_start >= start:  # If starts after the ref
                if c_end <= end:  # and end before, it is contained by the ref
                    mention_list[j].contained = mention_list[i].mention_id
                if c_start <= end:  # if starts before the end of the ref, only overlaps
                    mention_list[j].overlap = mention_list[i].mention_id

    for k in range(0, len(mention_list)):  # Filling missing info
        if not hasattr(mention_list[k], 'contained'):
            mention_list[k].contained = False
        if not hasattr(mention_list[k], 'overlap'):
            mention_list[k].overlap = False

    return mention_list


def _get_index(mentions):
    """
    Adds the 'index' and 'mention_position'.
    'index' is a counter for each mention in each document
    'mention_position' is the index in a [0,1] interval (0 is the first, 1 is the last).
    :param mentions: list of all mentions
    :return: the same list of input, with objects changed
    """
    doc_count = '0'
    count = 0
    i = 0
    mentions_in_each_doc = []
    for m in mentions:
        if m.doc_id == doc_count:  # Keep counting while it's the same doc
            count += 1
        else:
            mentions_in_each_doc.append(count)  # save the counter
            doc_count = m.doc_id
            count = 1
        m.index = count
    mentions_in_each_doc.append(count)  # save last counter

    # Now will transform into [0,1,...]
    doc_count = '0'
    for m in mentions:
        if m.doc_id == doc_count:
            m.mention_position = m.index / mentions_in_each_doc[i]
        else:
            doc_count = m.doc_id
            i += 1
            m.mention_position = m.index / mentions_in_each_doc[i]

    return mentions


def get_mention_pairs(train_list, increment_mention_info=None, increment_mention_pair=None,
                      use_pair=check_usable_pairs):
    """
    Builds a list of pair of mentions for the file. Each pair may or may not have a coreference. Each position
    simulates an object with the first two positions being mentions and the following are dictionaries with extra
    features

    :param train_list: list of lines in the file
    :param increment_mention_info: function to add more information to the mention
    :param increment_mention_pair: function to add more information to the mention pair
    :param use_pair: function to define if two mentions should be paired or not
    :return: list of objects
    """
    mention_list = build_mention_list(train_list, increment_mention_info)
    mention_pair_list = []
    for i in tqdm(range(1, len(mention_list)), desc="mention pair"):
        for j in range(0, i):
            if use_pair(mention_list, i, j):
                pair = MentionPair(mention_list[i], mention_list[j])
                mention_pair_list.append(pair)

    # Adding extra info
    mention_pair_list = _add_extra_pair_info(mention_pair_list, train_list, increment_mention_pair)

    return mention_pair_list


### loader

In [0]:
"""
This module will parse the files in the CoNLL format.
The main function is trainfile_to_vectors

"""

import os
import re

from tqdm import tqdm

# from boilerplate import mentions


def trainfile_to_vectors(path, increment_mention, increment_mention_pair, make_vectors):
    """
    Given one file, returns the input and output vectors to be passed to a learning algo
    :param path: file path to be used
    :param increment_mention: method to add information to the mention
    :param increment_mention_pair: method to add information to the mention pair
    :param make_vectors: method to build the vectors
    :return: [input_vector, output_vector, document_name]
    """
    train_list = train_file_to_list(path)
    pairs = mentions.get_mention_pairs(train_list, increment_mention, increment_mention_pair)
    input_vector, output_vector = make_vectors(pairs, train_list=train_list)
    input_vector = _append_mention_info(pairs, input_vector)
    return input_vector, output_vector, get_document_name(train_list)


def process_dir(path_in, path_out, callback):
    with tqdm(os.walk(path_in), desc="folders")as pb:
        for r, d, f in pb:
            pb.set_description("folder:...{}".format(r[-20:]))
            for file_name in tqdm(f, desc="files"):
                if not file_name.endswith("_conll"):
                    continue
                v_in, v_out, doc_name = callback(os.path.join(r, file_name))

                if len(v_in) > 0 and len(v_out) > 0:
                    _save_to_file(v_in, path_out, file_name + "_in", doc_name)
                    _save_to_file(v_out, path_out, file_name + "_out")


def transform_conll_to_vectors(path_in, path_out, increment_mention, increment_mention_pair, make_vectors):
    """
    Walks the input path looking for *_conll files. If any file is found, it is processed and two files are generated
    into the path_out root.
    A file with pattern [original_name]_in with the input vectors and [original_name]_out with the output vectors
    :param path_in: root folder to be searched. Files can be in multiple sub-folders
    :param path_out: output folder
    :param increment_mention: method to add information to the mention
    :param increment_mention_pair: method to add information to the mention pair
    :param make_vectors: method to build the vectors
    """

    process_dir(path_in, path_out,
                lambda x: trainfile_to_vectors(x, increment_mention, increment_mention_pair, make_vectors))


def train_file_to_list(file):
    """
    :param file: file name to be read
    :return: list of all lines
    """
    with open(file, "r", encoding="utf8") as f:
        return f.readlines()


def _save_to_file(vector, path, file_name, doc_name=None):
    """
    Saves a vector into a file if the vector is not empty
    :param vector: list of lists of values. Can be numpy arrays
    :param path: folder to save
    :param file_name: file name to use
    """
    if len(vector) == 0:  # Do not create empty files
        return
    with open(os.path.join(path, file_name), "w") as f:
        if doc_name:
            f.write(doc_name + "\n")

        for line in vector:
            f.write(",".join([str(i) for i in line]) + "\n")


def _append_mention_info(pairs, input_vectors):
    """
    Append pair information into the vector to be saved in the disk
    :param pairs: list of mention pairs
    :param input_vectors: list of vectors
    :return: a list of all information appended
    """
    appended = []
    for i in range(len(pairs)):
        vec_as_list = list([x[0] for x in input_vectors[i]])
        if len(vec_as_list) == 0:
            continue
        appended.append(list(pairs[i].get_info_vector()) + vec_as_list)

    return appended


def get_document_name(train_list):
    """
    The the document name from the first line.
    :param train_list: list of all lines in the document
    :return: the document name
    """
    name_re = re.compile(r".*\((.*)\).*")
    match = name_re.match(train_list[0])
    return match.group(1)

## pre_training

### create_bert_ds

In [0]:
import os
import numpy as np
#from boilerplate import loader
#from boilerplate import mentions
from tqdm import tqdm
#from boilerplate.mentions import CONLL_WORD_COLUMN
import random
import tensorflow as tf
import tensorflow_hub as hub
import bert
from bert import tokenization

MAX_SEQ_LEN = 512

BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"


class Span:
    def __init__(self, tokenizer, start_pos, words):
        self.start_pos = start_pos
        self.words = words
        self.end_pos = start_pos + len(words) - 1
        self.tokens = [tokenizer.tokenize(x) for x in words]
        self.tokens = [item for sublist in self.tokens for item in sublist]

        assert self.token_len() <= MAX_SEQ_LEN

    def token_len(self):
        return len(self.tokens)

    def __str__(self):
        return f"({self.start_pos}, {self.end_pos})"


class Cluster:
    def __init__(self):
        self.first_mention = None  # The first occurring mention
        self.mentions = []  # All other mentions in cluster

    def add_mention(self, mention):
        if mention in self.mentions:
            return

        if self.first_mention is None:  # First Mention
            self.first_mention = mention
        elif self.first_mention.compare(mention) > 0:
            self.mentions.append(self.first_mention)  # Save the mention that was the first
            self.first_mention = mention  # Set the new one
        else:
            self.mentions.append(mention)  # Just save the mention


class Line:
    def __init__(self, line):
        self.line = line
        self.word = None
        if isinstance(line, str):
            if ("#begin" in line) or ("#end" in line):
                return
            split = line.split()
            if len(split) >= CONLL_WORD_COLUMN:
                self.word = split[CONLL_WORD_COLUMN]

    def __str__(self):
        return self.line


class TreeNode:
    def __init__(self, mention, parent=None):
        self.mention = mention
        self.children = []
        self.parent = parent
        if self.parent:
            self.parent.add_node(self)

    def add_node(self, node):
        self.children.append(node)
        node.parent = self

    def add_mention(self, mention):
        node = TreeNode(mention)
        self.add_node(node)
        return node

    def delete(self):
        for c in self.children:
            self.parent.add_node(c)

        try:
            self.parent.children.remove(self)
        except ValueError:
            pass

    def get_mention_subtree(self):
        my_map = {}
        if self.mention is not None:
            my_map[self.mention.start_pos] = [self.mention.end_pos]

        for child in self.children:
            child_map = child.get_mention_subtree()
            for start, list_end in child_map.items():
                if start not in my_map:
                    my_map[start] = list_end
                else:
                    my_map[start] += list_end
        return my_map

    def __str__(self):
        return f"TreeNode ({self.parent}, {self.mention})"


class Tree:
    def __init__(self):
        self.root = TreeNode(None)
        self.mention_map = {}

    def add(self, mention, parent=None):
        if parent is None:
            return self.add(mention, self.root)

        return TreeNode(mention, parent)

    def get_parent_id(self, mention):
        if mention not in self.mention_map:
            print("Not in mapping")
            return 0

        node = self.mention_map[mention]
        parent = node.parent
        if parent is None or parent.mention is None:
            return -1
        return parent.mention.seq_id

    def update_mapping(self, new_map):
        self.mention_map.update(new_map)

    def remove_mention(self, mention):
        node = self.mention_map[mention]
        node.delete()

    def get_clusters(self):
        for child in self.root.children:
            yield child.get_mention_subtree()


class SuperStructure:
    def __init__(self, tokenizer, lines):
        self.structure_map = self.build_map(tokenizer, lines)
        self.size = len(lines)
        self.mentions = []

    def build_map(self, tokenizer, lines):
        all_groups = []
        for start, end in self.find_doc_spans(lines):
            spans = self.detect_spans(tokenizer, lines[start:end])
            span_grouped = self.group_spans(spans, MAX_SEQ_LEN)
            all_groups += [(x[0] + start, x[1] + start) for x in span_grouped]

        return {x: self.build_substructure(tokenizer, lines, x) for x in all_groups}

    @staticmethod
    def find_doc_spans(lines):
        start = -1
        for idx, line in enumerate(lines):
            if "#begin" in line:
                start = idx
            elif "#end" in line:
                yield (start, idx + 1)

    def add_mention(self, mention):
        self.mentions.append(mention)
        for span, structure in self.structure_map.items():
            if span[0] <= mention.start_pos and span[1] >= mention.end_pos:
                structure.add_mention(mention)

    def get_random_example(self, return_fist_pos=False):
        all_x1 = []
        all_x2 = []
        all_y = []
        first_pos = []
        for structure in self.structure_map.values():
            x1, x2, y = structure.get_random_example()
            all_x1.append(x1)
            all_x2.append(x2)
            all_y.append(y)
            first_pos.append(structure.first_position)

        output = [all_x1, all_x2, all_y]
        if return_fist_pos:
            output.append(first_pos)
        return output

    @staticmethod
    def build_substructure(tokenizer, lines, span):
        return Structure(tokenizer, lines[span[0]:(span[1] + 1)], span[0])

    @staticmethod
    def group_spans(spans, max_length):
        total_length = 0
        groups = []
        current_group = []

        def current_rep():
            start = current_group[0].start_pos
            end = current_group[-1].end_pos
            return start, end

        for span in spans:
            inc = span.token_len() + 1
            if total_length + inc > max_length:
                groups.append(current_rep())

                while total_length + inc > (max_length / 2) and len(current_group) > 0:
                    total_length -= current_group.pop(0).token_len() + 1

            current_group.append(span)
            total_length += inc

            assert total_length <= max_length, f"New length:{total_length}"

        if len(current_group) > 0:  # Last sentences
            assert sum([x.token_len() for x in current_group]) <= max_length
            groups.append(current_rep())

        return groups

    @staticmethod
    def detect_spans(tokenizer, lines):
        spans = []
        running = False
        start = 0
        words = []
        for idx, line in enumerate(lines):
            line = line.strip("\n").strip("\r")
            if "#begin" in line:
                continue
            if len(line) == 0 or "#end" in line:
                if running:
                    running = False
                    spans.append(Span(tokenizer, start, words))
                    words = []
                continue
            if not running:
                start = idx
                running = True
            words.append(line.split()[CONLL_WORD_COLUMN])

        return spans

    def __len__(self):
        return sum([len(s) for s in self.structure_map.values()])


class Structure:
    def __init__(self, tokenizer, lines, first_position):
        self.lines = [Line(x) for x in lines]
        self.clusters = {}
        self.flat = [[] for x in range(len(lines))]
        self.first_position = first_position
        self.tokens = [[] if l.word is None else tokenizer.tokenize(l.word) for l in self.lines]
        self.tokens = [tokenizer.convert_tokens_to_ids(t) for t in self.tokens]

        if sum([len(x) for x in self.tokens]) > MAX_SEQ_LEN:
            print("Too much")

    def __len__(self):
        return sum([len(x) for x in self.flat])

    def add_mention(self, mention):
        mention_id = mention.mention_id

        if mention_id not in self.clusters:
            self.clusters[mention_id] = Cluster()

        self.clusters[mention_id].add_mention(mention)
        self.flat[mention.start_pos - self.first_position].append(mention)

    def get_random_example(self):
        tree = self.get_random_tree()
        ids, parents = self.build_vectors(tree)

        X1 = []
        X2 = []
        Y = []

        for index, line in enumerate(self.lines):
            # if line.word is None:
            #    continue
            index_id = ids[index]
            parent_id = parents[index]
            for tk in self.tokens[index]:
                X1.append(tk)
                X2.append(index_id)
                Y.append(parent_id)

        assert len(Y) <= MAX_SEQ_LEN, f"Too big {len(Y)}, {self.first_position}"
        return X1, X2, Y

    def build_vectors(self, tree):
        ids = np.zeros(len(self.flat))
        parents = np.zeros(len(self.flat))

        counter = 1
        for i in range(len(ids)):
            mentions = self.flat[i]
            if len(mentions) == 0:
                continue
            mentions.sort(key=lambda x: x.end_pos)
            for mention in mentions:
                # mention = self.clean_inconsistency(mentions, tree)
                end_pos = mention.end_pos + 1
                ids[mention.start_pos: end_pos] = counter
                parents[mention.start_pos: end_pos] = tree.get_parent_id(mention)
                mention.seq_id = counter
                counter += 1
        return ids, parents

    def clean_inconsistency(self, mentions, tree):
        survivor = random.choice(mentions)
        for m in mentions:
            if m == survivor:
                continue
            tree.remove_mention(m)
        return survivor

    def get_random_tree(self):
        tree = Tree()
        for cluster in self.clusters.values():
            subtree, mapping = self.get_random_subtree(cluster)
            tree.root.add_node(subtree)
            tree.update_mapping(mapping)
        return tree

    @staticmethod
    def get_random_subtree(cluster):
        mapping = {}
        root = TreeNode(cluster.first_mention)
        mapping[cluster.first_mention] = root

        max_height = 0
        for mention in cluster.mentions:
            next_root = root
            height = 1
            last = round(random.random() * max_height)
            for i in range(last):
                # only descent in tree in children that occur before the current mention
                possible_children = [x for x in next_root.children if x.mention.compare(mention) < 0]

                if len(possible_children) == 0:
                    break
                next_root = random.choice(possible_children)
                height += 1
            node = next_root.add_mention(mention)
            mapping[mention] = node

            if height >= max_height:
                max_height = height

        return root, mapping


def build_structure(tokenizer, lines):
    structure = SuperStructure(tokenizer, lines)

    for mention in tqdm(build_mention_list(lines)):
        structure.add_mention(mention)

    return structure


def build_mention_list(lines):
    return build_mention_list_boilerplate(lines)


def build_mention_list_boilerplate(lines):
    return mentions.build_mention_list(lines)


def build_mention_list_cort(lines):
    from cort.core import corpora
    from cort.core import mention_extractor

    document_as_strings = []

    current_document = ""

    for line in lines:
        if line.startswith("#begin") and current_document != "":
            document_as_strings.append(current_document)
            current_document = ""
        current_document += line

    document_as_strings.append(current_document)

    training_corpus = corpora.Corpus("test", sorted([corpora.from_string(doc) for
                                                     doc in document_as_strings]))
    for idx, doc in enumerate(training_corpus):
        for mention in mention_extractor.extract_system_mentions(doc):
            # print(f"New Mention:{mention}")

            if mention.span is None:
                continue

            words = mention.attributes["tokens"]
            start_pos = mention.span.begin
            end_pos = mention.span.end
            my_mention = Mention(f"{idx}_A", words, start_pos, end_pos)
            yield my_mention


def to_vectors(file_path, num_examples, tokenizer):
    lines = loader.train_file_to_list(file_path)
    structure = build_structure(tokenizer, lines)

    X1 = []
    X2 = []
    Y = []

    if len(structure) == 0:
        return X1, X2, Y

    found = set()
    for i in tqdm(range(num_examples * 3)):
        x1, x2, y = structure.get_random_example()
        ex_id = sum([sum(k) for k in y])
        if ex_id in found:
            continue
        found.add(ex_id)
        X1 += x1
        X2 += x2
        Y += y
        if len(found) >= num_examples:
            break

    return X1, X2, Y


def create_tokenizer():
    """Get the vocab file and casing info from the Hub module."""
    with tf.Graph().as_default():
        bert_module = hub.Module(BERT_MODEL_HUB)
        tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
        with tf.Session() as sess:
            vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                                  tokenization_info["do_lower_case"]])

    return bert.tokenization.FullTokenizer(
        vocab_file=vocab_file, do_lower_case=do_lower_case)

### fine_tune_bert

In [0]:
import tensorflow as tf

import bert
from bert import optimization
import tensorflow_hub as hub
import sys
import tf_metrics

hooks = []
debug = {}

root = r"/content/gdrive/My Drive/Puc/Projeto Final/"
train_data_folder = f"{root}/Datasets/finetuning/devel/"
predict_data_folder = f"{root}/Datasets/conll/conll-2012-test/"

OUTPUT_DIR = f"{root}/models/bert/devel_ok"

BERT_MODEL_HUB = "https://tfhub.dev/google/bert_multi_cased_L-12_H-768_A-12/1"
# Compute train and warmup steps from batch size
BATCH_SIZE = 4
MAX_EXAMPLES = 2000
LEARNING_RATE = 2e-5
# Warmup is a period of time where hte learning rate
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100
MAX_SEQ_LEN = 512
num_classes = 768
pos_indices = list(range(1, num_classes))
average = 'micro'
features_path = sys.argv[-1]
MAX_CLASSES = 120
min_loss = 1000
NON_ROOT_WEIGHT = 3

tf.logging.set_verbosity(tf.logging.INFO)


def build_estimator(total_samples, num_epochs):
    # Compute # train and warmup steps from batch size
    num_train_steps = int(total_samples * num_epochs / BATCH_SIZE)
    num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)
    # Create an input function for training. drop_remainder = True for using TPUs.

    # Specify outpit directory and number of checkpoint steps to save
    run_config = tf.estimator.RunConfig(
        model_dir=OUTPUT_DIR,
        save_summary_steps=SAVE_SUMMARY_STEPS,
        save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

    model_fn = model_fn_builder(
        num_labels=0,
        learning_rate=LEARNING_RATE,
        num_train_steps=num_train_steps,
        num_warmup_steps=num_warmup_steps)

    estimator = tf.estimator.Estimator(
        model_fn=model_fn,
        config=run_config,
        params={"batch_size": BATCH_SIZE})

    return estimator


def input_fn_builder(tokens, ids, labels, mask, is_training, drop_remainder, loss_weight):
    """Creates an `input_fn` closure to be passed to TPUEstimator."""

    # print(f"Tokens shape:{tokens.shape}")

    num_examples, seq_length = tokens.shape

    # print(f"t {num_examples}, {seq_length}")

    all_input_ids = tokens
    all_input_mask = mask
    all_segment_ids = ids
    all_label_ids = labels

    def input_fn(params):
        """The actual input function."""
        batch_size = params["batch_size"]

        # This is for demo purposes and does NOT scale to large data sets. We do
        # not use Dataset.from_generator() because that uses tf.py_func which is
        # not TPU compatible. The right way to load data is with TFRecordReader.
        d = tf.data.Dataset.from_tensor_slices({
            "input_ids": all_input_ids,
            "input_mask": all_input_mask,
            "segment_ids": all_segment_ids,
            "label_ids": all_label_ids,
            "loss_weight": loss_weight
        })

        if is_training:
            # d = d.repeat()
            d = d.shuffle(buffer_size=100)

        d = d.batch(batch_size=batch_size, drop_remainder=drop_remainder)
        d = d.prefetch(batch_size)
        return d

    return input_fn


def create_model(is_predicting, input_ids, input_mask, segment_ids, labels, loss_weight):
    """Creates a classification model."""

    bert_module = hub.Module(
        BERT_MODEL_HUB,
        trainable=True)
    bert_inputs = dict(
        input_ids=input_ids,
        input_mask=input_mask,
        segment_ids=segment_ids)
    bert_outputs = bert_module(
        inputs=bert_inputs,
        signature="tokens",
        as_dict=True)

    one_hot_labels = tf.one_hot(labels, MAX_CLASSES, name="My_OneHot")

    # Use "pooled_output" for classification tasks on an entire sentence.
    # Use "sequence_outputs" for token-level output.
    output_layer = bert_outputs["sequence_output"]
    hidden_size = output_layer.shape[-1].value

    output_weights = tf.get_variable(
        "output_weights", [MAX_CLASSES, hidden_size],
        initializer=tf.truncated_normal_initializer(mean=0.0, stddev=1))

    output_bias = tf.get_variable(
        "output_bias", [MAX_SEQ_LEN, MAX_CLASSES], initializer=tf.zeros_initializer())

    with tf.variable_scope("loss"):
        output_layer_2 = tf.matmul(output_layer, output_weights, transpose_b=True)
        output_layer_2 = tf.add(output_layer_2, output_bias)
        # output_layer_2 = tf.layers.batch_normalization(output_layer_2, training=(not is_predicting))
        # output_layer_2 = tf.math.sigmoid(output_layer_2, name="My_output_layer")
        output_layer_2 = tf.nn.relu(output_layer_2, name="My_output_layer")

        # Dropout helps prevent overfitting
        # output_layer = tf.nn.dropout(output_layer, keep_prob=0.9, name="My_Dropout")
        # print(f"1 output_layer shape:{output_layer.shape}")

        # output_layer = tf.nn.softmax(output_layer, name="My Softmax")
        # print(f"2 output_layer shape:{output_layer.shape}")

        # tmp_mask = tf.broadcast_to(tf.expand_dims(input_mask, axis=-1), tf.shape(output_layer))
        # output_layer = tf.math.multiply(output_layer, tf.cast(tmp_mask, tf.float32), name="My_Argmax_Mask")

        # softmax = tf.nn.softmax(output_layer_2, name="My_Softmax")
        argmax = tf.math.argmax(output_layer_2, axis=-1, name="My_Argmax")

        predicted_labels = tf.math.multiply(argmax, tf.cast(input_mask, tf.int64), name="d")

        hooks.append(
            tf.estimator.LoggingTensorHook({
                # "softmax": softmax[0][0],
                # "label": labels[0],
                # "argmax": argmax[0][0],
                # "segment_ids": segment_ids[0],
                # "output_layer": output_layer[0][0],
                "predicted": predicted_labels[0],
                "output_layer_2": output_layer_2[0][0],
                "output_weights": output_weights[0],
                "output_bias": output_bias[0],
                # "OK labels": tf.math.reduce_sum(
                #    tf.to_float(tf.math.equal(tf.cast(labels, tf.int64), predicted_labels))),
                # "Total Labels": tf.shape(labels),
                # "softmax shape": tf.shape(softmax),
                # "argmax shape": tf.shape(argmax),
                # "predicted_labels shape": tf.shape(predicted_labels),
                # "input_mask shape": tf.shape(input_mask)
            }, every_n_iter=10))

        if is_predicting:
            return predicted_labels

        # output_layer = tf.reshape(output_layer, [BATCH_SIZE ,MAX_SEQ_LEN, 768])
        # print(f"logits : {output_layer.shape}")
        # print(f"tmp : {tmp.shape}")

        loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=one_hot_labels, logits=output_layer_2,
                                                          name="My_softmax_loss")
        no_loss = tf.multiply(loss, tf.cast(input_mask, tf.float32), name="My_masked_loss")
        weightened = tf.divide(no_loss, tf.cast(tf.reduce_sum(input_mask), tf.float32), name="My_wloss")
        rm = tf.reduce_sum(weightened, name="My_final_loss")

        hooks.append(
            tf.estimator.LoggingTensorHook({
                # "count no_loss": tf.math.count_nonzero(loss),
                # "count input_mask": tf.math.count_nonzero(input_mask),
                # "loss shape": tf.shape(loss),
                # "rm shape": tf.shape(rm),
                # "no_loss shape": tf.shape(no_loss),
                # "one_hot_labels shape": tf.shape(one_hot_labels),
                "no_loss": no_loss[0],
                # "weightened": weightened[0],
                "loss_weight": loss_weight[0],
                "final_loss": rm
            }, every_n_iter=10))

        return rm, predicted_labels


# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
    """Returns `model_fn` closure for TPUEstimator."""

    def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
        """The `model_fn` for TPUEstimator."""

        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        segment_ids = features["segment_ids"]
        label_ids = features["label_ids"]
        loss_weight = features["loss_weight"]

        is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)

        # TRAIN and EVAL
        if not is_predicting:

            (loss, predicted_labels) = create_model(
                is_predicting, input_ids, input_mask, segment_ids, label_ids, loss_weight)

            hooks.clear()
            hooks.append(tf.train.LoggingTensorHook({"loss": loss}, every_n_iter=10))

            train_op = bert.optimization.create_optimizer(
                loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

            # Calculate evaluation metrics.
            def metric_fn(label_ids, predicted_labels):
                y_true, y_pred = label_ids, predicted_labels

                precision = tf_metrics.precision(
                    y_true, y_pred, num_classes, pos_indices, average=average)
                recall = tf_metrics.recall(
                    y_true, y_pred, num_classes, pos_indices, average=average)
                f2 = tf_metrics.fbeta(
                    y_true, y_pred, num_classes, pos_indices, average=average, beta=2)
                f1 = tf_metrics.f1(
                    y_true, y_pred, num_classes, pos_indices, average=average)

                return {
                    "precision": precision,
                    "recall": recall,
                    "f2": f2,
                    "f1": f1
                }

            eval_metrics = metric_fn(label_ids, predicted_labels)

            if mode == tf.estimator.ModeKeys.TRAIN:

                return tf.estimator.EstimatorSpec(mode=mode,
                                                  loss=loss,
                                                  train_op=train_op,
                                                  training_hooks=hooks)
            else:
                return tf.estimator.EstimatorSpec(mode=mode,
                                                  loss=loss,
                                                  eval_metric_ops=eval_metrics)
        else:
            predicted_labels = create_model(
                is_predicting, input_ids, input_mask, segment_ids, label_ids, loss_weight)

            return tf.estimator.EstimatorSpec(mode, predictions=predicted_labels)

    # Return the actual model function in the closure
    return model_fn


## predict_fine_tune

In [0]:
#from boilerplate.saver import save_documents, Document
#from pre_training import create_bert_ds, fine_tune_bert
#from boilerplate.loader import get_document_name
import numpy as np
import gc


def slice_input(tokens, ids, labels):
    print("Slicing")

    def fix_it(df):
        df = [x + ([np.nan] * (MAX_SEQ_LEN - len(x))) for x in df]
        df = np.array(df)
        return np.nan_to_num(df, False).astype('int32')

    sliced_labels = fix_it(labels)
    sliced_tokens = fix_it(tokens)
    sliced_ids = fix_it(ids)
    sliced_mask = (sliced_ids > 0).astype('int32')

    sliced_loss_weight = (sliced_ids > 1) * (NON_ROOT_WEIGHT - 1)  # Will add 1 in row above
    sliced_loss_weight += sliced_mask

    print("Sliced")
    return input_fn_builder(sliced_tokens, sliced_ids, sliced_labels, sliced_mask, False, False,
                                           sliced_loss_weight)


def detect_mentions(lines):
    return build_mention_list_cort(lines)


def predict_example(example, estimator):
    tokens, ids, labels, first_pos = example
    predict_input_fn = slice_input(tokens, ids, labels)
    predictions = estimator.predict(predict_input_fn)
    return [x-1 for x in predictions] # -1 por convencao do labelø


def get_single_seq(struct, parents):
    for mention in struct.mentions:
        mention_parents = parents[mention.start_pos:(mention.end_pos + 1)]
        if len(mention_parents) == 0:
            print("Empty mention")
            continue

        mention.parent_id = int(max(mention_parents))

    return struct.mentions


def find_clusters(mention_seq):
    tree = Tree()
    mention_seq.sort(key=lambda x : x.mention_id)
    nodes = {}

    for idx, mention in enumerate(mention_seq):        
        parent = nodes[int(mention.parent_id)] if mention.parent_id in nodes else tree.root
        nodes[mention.mention_id] = tree.add(mention, parent)

    return tree.get_clusters()


def fill_doc(struct, parents, doc):
    mention_seq = get_single_seq(struct, parents)
    clusters = find_clusters(mention_seq)
    for c in clusters:
        doc.add_cluster(c)


def predict_doc(lines, first_line_idx, tokenizer, estimator):
    struct = build_struct(lines, tokenizer)

    example = struct.get_random_example(True)
    predicted = predict_example(example, estimator)

    parents = merge_predictions(predicted, struct)

    doc = Document(get_document_name(lines), first_line_idx)
    fill_doc(struct, parents, doc)
    return doc


def merge_predictions(predicted, struct):
    parents = [0] * struct.size
    for idx, sub_struct in enumerate(struct.structure_map.values()):
        current_predictions = predicted[idx]
        parent_runner = sub_struct.first_position
        struct_runner = 0
        for tk in sub_struct.tokens:
            tk_len = len(tk)
            predicted_parenst = current_predictions[struct_runner:(struct_runner + tk_len)]
            if len(predicted_parenst) == 0:
                struct_runner += 1
                parent_runner += 1
                continue
            parent_id = round(np.mean(predicted_parenst))
            parents[parent_runner] = max(parent_id, parents[parent_runner])
            struct_runner += tk_len
            parent_runner += 1
    return parents


def build_struct(lines, tokenizer):
    struct = SuperStructure(tokenizer, lines)
    for mention in detect_mentions(lines):
        struct.add_mention(mention)
    return struct


def split_in_docs(file_name):
    buffer = []
    with open(file_name, "r") as f:
        for line in f:
            buffer.append(line)
            if "#end" in line:
                yield buffer
                buffer = []


def predict_docs_in_file(file_name, tokenizer, estimator):
    line_counter = 0
    for doc_lines in split_in_docs(file_name):
        yield predict_doc(doc_lines, line_counter, tokenizer, estimator)
        line_counter += len(doc_lines)
        gc.collect()


def save_docs_to_file(in_file, out_file, docs):
    save_documents(in_file, out_file, docs)


# Main

In [0]:
predicted_docs = predict_docs_in_file(test_file,
                                      tokenizer=create_tokenizer(),
                                      estimator=build_estimator(10, 1))
save_docs_to_file(test_file, predicted_file, predicted_docs)


I0811 16:21:11.260418 139751407007616 saver.py:1499] Saver not created because there are no variables in the graph to restore
W0811 16:21:12.065522 139751407007616 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/bert/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

I0811 16:21:12.602044 139751407007616 estimator.py:209] Using config: {'_model_dir': '/content/gdrive/My Drive/Puc/Projeto Final//models/bert/devel_ok', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': No

Slicing
Sliced


I0811 16:21:22.456465 139751407007616 saver.py:1499] Saver not created because there are no variables in the graph to restore
I0811 16:21:22.679583 139751407007616 estimator.py:1147] Done calling model_fn.
W0811 16:21:22.872650 139751407007616 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/array_ops.py:1354: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
I0811 16:21:23.337346 139751407007616 monitored_session.py:240] Graph was finalized.
W0811 16:21:23.339201 139751407007616 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for file

Writing doc tc/ch/00/ch_0039


I0811 16:22:08.011440 139751407007616 estimator.py:1145] Calling model_fn.


...

I0811 16:53:31.486509 139751407007616 estimator.py:1145] Calling model_fn.


Slicing
Sliced


I0811 16:53:34.483219 139751407007616 saver.py:1499] Saver not created because there are no variables in the graph to restore
I0811 16:53:34.717461 139751407007616 estimator.py:1147] Done calling model_fn.
I0811 16:53:35.323499 139751407007616 monitored_session.py:240] Graph was finalized.
I0811 16:53:35.367443 139751407007616 saver.py:1280] Restoring parameters from /content/gdrive/My Drive/Puc/Projeto Final//models/bert/devel_ok/model.ckpt-8290
I0811 16:54:09.139487 139751407007616 session_manager.py:500] Running local_init_op.
I0811 16:54:10.943263 139751407007616 session_manager.py:502] Done running local_init_op.


Moving to next document
Writing doc bn/cnn/00/cnn_0019
Slicing
Sliced


I0811 16:54:57.748317 139751407007616 estimator.py:1145] Calling model_fn.
I0811 16:55:00.770701 139751407007616 saver.py:1499] Saver not created because there are no variables in the graph to restore
I0811 16:55:01.006092 139751407007616 estimator.py:1147] Done calling model_fn.
I0811 16:55:01.608205 139751407007616 monitored_session.py:240] Graph was finalized.
I0811 16:55:01.721855 139751407007616 saver.py:1280] Restoring parameters from /content/gdrive/My Drive/Puc/Projeto Final//models/bert/devel_ok/model.ckpt-8290
